In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torchtext import data
import re 
import numpy as np
# nltk.download('stopwords')
# label = pd.read_csv('label.csv') # Labelled Data provided
# label['id'] = label['id'].apply(lambda x: x.split('-')[0]).astype(int)

In [3]:
def cleanup_text(texts):
    text = texts
    # en_stops = set(stopwords.words('english'))
    if(texts == np.nan):
        return np.nan
    if (len(texts) < 20):
        return np.nan
    # Lower case
    text = text.lower()
    #  # remove punctuation
    # text = re.sub('[^a-zA-Z0-9]', ' ', text)

    # remove newline
    text = re.sub(r'\n', ' ', text)    # remove multiple spaces
    text = re.sub(r' +', ' ', text)
    # Remove stopwords
    # filtered_words = [word for word in text.split(' ') if word not in stopwords.words('english')]
    return text

# Preprocess Sentence data

In [4]:
sentence_data = pd.read_csv('sentence_new.csv') # first labelled
# sentence_data2 = pd.read_csv('label2.csv') # the data u just gave me

In [3]:
# sentence_data = pd.concat([sentence_data,sentence_data2])

In [5]:
sentence_data['target'].value_counts()

1    136
0    134
Name: target, dtype: int64

In [6]:
sentence_data = sentence_data.rename(columns={"target": "labels","sentence":"text"})

In [7]:
sentence_data['text'] = sentence_data['text'].apply(cleanup_text)

# Split data
validation_percent = 0.20

# Split data into train / validate
train,test = train_test_split(sentence_data,test_size=validation_percent,random_state = 10)

In [8]:
train[train['text'].isna()]

,index,text,labels


In [9]:
test[test['text'].isna()]

,index,text,labels


In [10]:
train.to_csv('train.csv',index = False)
test.to_csv('test.csv', index=False)

In [13]:
for i,row in sentence_data.iterrows():
    if(row['target'] == 1):
        print(row['sentence'])
        print('end ...')
    # print(len(row['sentence']))

she has + bowel sounds but no stool as yet.house staff waiting on placing a ft for now as she was very uncooperative earlier.her bs are improved on current ss and fixed dosing.
end ...
his day consists of going for a number of walks with intervals of [**location (un) 308**] personality magazines, and playing his guitar.one way he has found for dealing with his anger is listing to a [**doctor last name **] group from ca because they are as angry as he is.most of his anger is directed at the negative state of the world.
end ...
 ls clear at apices to diminished at bases, rr 11-22, o2 sats 98-100% on 1.5 l n.c. neuro pt alert and oriented x 3, pleasant and cooperative with care, asking appropriate questions regarding care, mae.pt oob to commode with supervision, pt bp tol.
end ...
discussed that prognosis at this time is poor in regards to neurological outcome.per family, at baseline, patient was very active (played golf, went dancing, went to the gym) and he would not want to have extrao

# Postprocess predict data

In [66]:

import pandas as pd
sentence_data = pd.read_csv('processed_patients.csv')

/tmp/ipykernel_1440/155315861.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  sentence_data = pd.read_csv('processed_patients.csv')


In [54]:
result = pd.read_csv('result.csv')

In [60]:
info_data = sentence_data.drop('TEXT',axis = 1).drop_duplicates()

In [62]:
info_data

,SUBJECT_ID,GENDER,ETHNICITY,AGE,simple_ethnicity
0,3,M,WHITE,76.0,WHITE
306,4,F,WHITE,48.0,WHITE
367,6,F,WHITE,66.0,WHITE
441,9,M,UNKNOWN/NOT SPECIFIED,41.0,UNKNOWN
460,11,F,WHITE,50.0,WHITE
...,...,...,...,...,...
3076091,98526,F,UNABLE TO OBTAIN,55.0,UNABLE
3076187,98530,M,WHITE,75.0,WHITE
3076319,98531,M,WHITE,77.0,WHITE
3076345,98552,F,WHITE,89.0,WHITE


In [63]:
result['id'] = result['id'].astype(int)
final = result.merge(info_data ,left_on='id',right_on='SUBJECT_ID',how='left')

In [65]:
final.to_csv('result.csv',index = False)

In [7]:
import numpy as np
import re

In [67]:
sentence_data.head()

,SUBJECT_ID,GENDER,ETHNICITY,AGE,simple_ethnicity,TEXT
0,3,M,WHITE,76.0,WHITE,PATIENT/TEST INFORMATION:\nIndication: Left ve...
1,3,M,WHITE,76.0,WHITE,Height: (in) 70\nWeight (lb): 215\nBSA (m2): 2...
2,3,M,WHITE,76.0,WHITE,RIGHT ATRIUM/INTERATRIAL SEPTUM: The right atr...
3,3,M,WHITE,76.0,WHITE,LEFT VENTRICLE: Left ventricular wall thicknes...
4,3,M,WHITE,76.0,WHITE,The left\nventricular cavity size is normal.


In [14]:
sentence_data = sentence_data.dropna()
sentence_data = sentence_data.rename(columns={"TEXT": "text"})
sentence_data['text'] = sentence_data['text'].apply(cleanup_text)

In [15]:
sentence_data = sentence_data.dropna()

In [46]:
patients = sentence_data['SUBJECT_ID'].unique()
final_df = pd.DataFrame(columns=['id','text'])

In [47]:
for id in patients:
    test_df = sentence_data[sentence_data.SUBJECT_ID == id].reset_index()
    temp = test_df.groupby(test_df.index // 3)['text'].agg(' '.join).to_frame()
    temp['id'] = id
    final_df = pd.concat([final_df, temp], ignore_index=True)

KeyboardInterrupt: 

In [52]:
final_df.to_csv('eval_data_age_gender.csv',index = False)

In [ ]:
from tqdm import tqdm
for i in tqdm(len(patients)):
    id = patients[i]
    test_df = sentence_data[sentence_data.SUBJECT_ID == id].reset_index()
    temp = test_df.groupby(test_df.index // 3)['text'].agg(' '.join).to_frame()
    temp['id'] = id
    final_df = pd.concat([final_df, temp], ignore_index=True)